## Import packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from helper import series_to_supervised
from tensorflow.keras.models import load_model
from preprocess import water_postprocess
from scipy.stats import mannwhitneyu, wilcoxon, ttest_rel
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from performance import metrics_s1_t1

2024-02-03 15:15:43.213301: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

## Preprocess data

In [4]:
# https://keras.io/examples/timeseries/timeseries_weather_forecasting/#climate-data-timeseries
data = pd.read_csv("../data/jena_climate_2009_2016_hourly.csv", index_col=0)
data.fillna(0, inplace=True)
data.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:00:00,996.528000,-8.304000,265.118000,-9.120000,93.780000,3.260000,3.058000,0.202000,1.910000,3.068000,1309.196000,0.520000,1.002000,174.460000
2009-01-01 01:00:00,996.525000,-8.065000,265.361667,-8.861667,93.933333,3.323333,3.121667,0.201667,1.951667,3.133333,1307.981667,0.316667,0.711667,172.416667
2009-01-01 02:00:00,996.745000,-8.763333,264.645000,-9.610000,93.533333,3.145000,2.940000,0.201667,1.836667,2.950000,1311.816667,0.248333,0.606667,196.816667
2009-01-01 03:00:00,996.986667,-8.896667,264.491667,-9.786667,93.200000,3.111667,2.898333,0.210000,1.811667,2.906667,1312.813333,0.176667,0.606667,157.083333
2009-01-01 04:00:00,997.158333,-9.348333,264.026667,-10.345000,92.383333,3.001667,2.775000,0.231667,1.733333,2.780000,1315.355000,0.290000,0.670000,150.093333


In [5]:
values = data.values

# specify the number of lag hours
n_hours = 24*3
n_features = data.shape[-1]
k = 12
split1 = 0.7
split2 = 0.85

# frame as supervised learning
reframed = series_to_supervised(values, n_hours, k)
print("reframed.shape:", reframed.shape)

reframed.shape: (70046, 1176)


In [6]:
# split into train and test sets
reframed_values = reframed.values
n_train_hours = int(len(reframed_values)*split1)
n_valid_hours = int(len(reframed_values)*split2)

train = reframed_values[:n_train_hours, :]
val = reframed_values[n_train_hours:n_valid_hours, :]
test = reframed_values[n_valid_hours:, :]


# split into input and outputs
n_obs = n_hours * n_features
feature_idx = 5
train_X, train_y = train[:, :n_obs], train[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
val_X, val_y = val[:, :n_obs], val[:, [n_obs + feature_idx + n_features * i for i in range(k)]]
test_X, test_y = test[:, :n_obs], test[:, [n_obs + feature_idx + n_features * i for i in range(k)]]


print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (49032, 1008) (49032, 12) (10507, 1008) (10507, 12) (10507, 1008) (10507, 12)


In [7]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)

val_X = scaler.fit_transform(val_X)
val_y = scaler.fit_transform(val_y)

test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
val_X = val_X.reshape((val_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print("train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape", 
      train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape
     )

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape (49032, 72, 14) (49032, 12) (10507, 72, 14) (10507, 12) (10507, 72, 14) (10507, 12)


### Pressure threshold

In [8]:
train_X_pm = train_X[:, 0, feature_idx]
print(train_X_pm.shape)

val_X_pm = val_X[:, 0, feature_idx]
print(val_X_pm.shape)

test_X_pm = test_X[:, 0, feature_idx]
print(test_X_pm.shape)

(49032,)
(10507,)
(10507,)


In [9]:
percentile = 95

merged_array = np.concatenate((train_X_pm, val_X_pm, test_X_pm))

percentile_pm = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_pm)

95th Percentile of Daily Rain: 0.48441203148316114


### train_X_filter

In [10]:
train_X_extreme = train_X[train_X_pm > percentile_pm]
print(train_X_extreme.shape)

train_y_extreme = train_y[train_X_pm > percentile_pm]
print(train_y_extreme.shape)

(2206, 72, 14)
(2206, 12)


In [11]:
train_X_normal = train_X[train_X_pm <= percentile_pm]
print(train_X_normal.shape)

train_y_normal = train_y[train_X_pm <= percentile_pm]
print(train_y_normal.shape)

(46826, 72, 14)
(46826, 12)


### val_X_filter

In [12]:
val_X_extreme = val_X[val_X_pm > percentile_pm]
print(val_X_extreme.shape)

val_y_extreme = val_y[val_X_pm > percentile_pm]
print(val_y_extreme.shape)

(486, 72, 14)
(486, 12)


In [13]:
val_X_normal = val_X[val_X_pm <= percentile_pm]
print(val_X_normal.shape)

val_y_normal = val_y[val_X_pm <= percentile_pm]
print(val_y_normal.shape)

(10021, 72, 14)
(10021, 12)


### test_X_filter

In [14]:
test_X_extreme = test_X[test_X_pm > percentile_pm]
print(test_X_extreme.shape)

test_y_extreme = test_y[test_X_pm > percentile_pm]
print(test_y_extreme.shape)

(811, 72, 14)
(811, 12)


In [15]:
test_X_normal = test_X[test_X_pm <= percentile_pm]
print(test_X_normal.shape)

test_y_normal = test_y[test_X_pm <= percentile_pm]
print(test_y_normal.shape)

(9696, 72, 14)
(9696, 12)


## Test model

In [16]:
# ws_threshold = 2.58
time_index = 0

#### Extreme

In [23]:
# saved_model = load_model('../saved_models_mlp/pressure_N.h5') 
# saved_model = load_model('../saved_models_mlp/pressure_E.h5') 


# saved_model = load_model('../saved_models_mlp/pressure_all.h5') 
# saved_model = load_model('../saved_models_mlp/pressure_all_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_IPF_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_IPF_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_EVT_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_EVT_95_ft.h5')


saved_model = load_model('../saved_models_mlp/pressure_all_weighted_META_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_META_95_ft.h5')


yhat_extreme = saved_model.predict(test_X_extreme)

inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme = inv_yhat_extreme - inv_y_extreme

# print('MAE = {}'.format(float("{:.6f}".format(mae(inv_y_extreme, inv_yhat_extreme)))))
# print('RMSE = {}'.format(float("{:.6f}".format(np.sqrt(mse(inv_y_extreme, inv_yhat_extreme))))))

In [24]:
metrics_s1_t1(inv_y_extreme.min(), time_index, inv_y_extreme, inv_yhat_extreme, test_errors_extreme)

Peformance when water level is over 3.27 ft 

------ MAE & RMSE ------
MAE = 3.219878
RMSE = 4.457376 

------ Max Errors (t+1 at S1) ------
Max Error of Over Estimation: 10.44469217936198
Max Error of Under Estimation: -11.982028503417972
Max Abs Error of Under Estimation: 11.982028503417972 

------ Time # (t+1 at S1) ------
Time# of Over Estimation: 400
Time# of Under Estimation: 411 

------ Area (t+1 at S1) ------
Area of Over Estimation: 798.4090934054059
Area of Under Estimation: -1081.5422036616008


#### Normal & Extreme hen water level is over threshold 2.58 feet (95 percentile)

In [22]:
# saved_model = load_model('../saved_models_mlp/pressure_all.h5') 
# saved_model = load_model('../saved_models_mlp/pressure_all_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_IPF_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_IPF_95_ft.h5')


saved_model = load_model('../saved_models_mlp/pressure_all_weighted_EVT_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_EVT_95_ft.h5')


# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_META_95.h5')
# saved_model = load_model('../saved_models_mlp/pressure_all_weighted_META_95_ft.h5')


yhat = saved_model.predict(test_X)


inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)
test_errors = inv_yhat - inv_y

metrics_s1_t1(inv_y.min(), time_index, inv_y, inv_yhat, test_errors)

Peformance when water level is over 0.0 ft 

------ MAE & RMSE ------
MAE = 2.281178
RMSE = 3.400608 

------ Max Errors (t+1 at S1) ------
Max Error of Over Estimation: 14.390753784179687
Max Error of Under Estimation: -14.263462613423663
Max Abs Error of Under Estimation: 14.390753784179687 

------ Time # (t+1 at S1) ------
Time# of Over Estimation: 668
Time# of Under Estimation: 9766 

------ Area (t+1 at S1) ------
Area of Over Estimation: 609.5597838433589
Area of Under Estimation: -15401.501259616178


In [20]:
# metrics_s1_t1(inv_y.min(), time_index, inv_y, inv_yhat, test_errors)

#### hyperparameter - frozen layers

In [35]:
layers = [1, 5, 9, 13, 17]

for layer in layers:
    print('layer: {}'.format(layer))    
    saved_model = load_model('../saved_models_hyper/pressure_all_weighted_META_ft_{}.h5'.format(layer))
    yhat_extreme = saved_model.predict(test_X_extreme)

    inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
    inv_y_extreme = scaler.inverse_transform(test_y_extreme)
    test_errors_extreme = inv_yhat_extreme - inv_y_extreme

    print('MAE = {}'.format(float("{:.6f}".format(mae(inv_y_extreme, inv_yhat_extreme)))))
    print('RMSE = {}'.format(float("{:.6f}".format(np.sqrt(mse(inv_y_extreme, inv_yhat_extreme))))))
    print('-------------------------')

layer: 1
MAE = 3.061688
RMSE = 4.032437
-------------------------
layer: 5
MAE = 2.668814
RMSE = 3.78279
-------------------------
layer: 9
MAE = 3.071883
RMSE = 4.114493
-------------------------
layer: 13
MAE = 3.134275
RMSE = 4.205468
-------------------------
layer: 17
MAE = 3.094477
RMSE = 4.143517
-------------------------


### p-values

In [19]:
saved_model = load_model('../saved_models_mlp/pressure_all.h5') 

yhat_extreme = saved_model.predict(test_X_extreme)
inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme_ori = inv_yhat_extreme - inv_y_extreme
test_errors_extreme_ori.shape

(811, 12)

In [20]:
saved_model = load_model('../saved_models_mlp/pressure_all_weighted_META_95.h5')

yhat_extreme = saved_model.predict(test_X_extreme)
inv_yhat_extreme = scaler.inverse_transform(yhat_extreme)
inv_y_extreme = scaler.inverse_transform(test_y_extreme)
test_errors_extreme_re_meta = inv_yhat_extreme - inv_y_extreme

In [21]:
t_index = -1

# ========= Mann-Whitney U test =========
stat_mann, p_value_mann = mannwhitneyu(test_errors_extreme_ori[:,t_index], test_errors_extreme_re_meta[:,t_index], alternative='two-sided')
print(f"p_value_mann: {p_value_mann:.4e}")


# ========= wilcoxon U test =========
stat_wilcoxon, p_value_wilcoxon = wilcoxon(test_errors_extreme_ori[:, t_index], test_errors_extreme_re_meta[:, t_index])
print(f"p_value_wilcoxon: {p_value_wilcoxon:.4e}")


# ========= t-test =========
t_statistic, p_value = ttest_rel(test_errors_extreme_ori[:,t_index], test_errors_extreme_re_meta[:,t_index])
print(f"p_value_ttest: {p_value:.4e}")

p_value_mann: 3.1163e-06
p_value_wilcoxon: 7.5404e-47
p_value_ttest: 3.8191e-51
